In [1]:
# dependencies
import pymongo as pm
import urllib
from scrape import Scrape as sc
from pprint import pprint
from bs4 import BeautifulSoup as bs
import re
import time

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pm.MongoClient(conn)

In [3]:
# define db and collections
db = client.scrape_db

# this collection will store all the HTML pages
htmlCollection = db.HTML_pages

In [4]:
# startURL = 'http://books.toscrape.com/'
# domain = 'toscrape.com'

# declare top-level url and domain
startURL = 'https://billypenn.com/'
domain = 'billypenn.com'

# establish necessary data structures
done_urls = set()
external_urls = []
urls = set()
pagelinks = set()

# seed with top-level url of website
urls.add(startURL)

In [5]:
# counter for limiting functions
ctr = 0

# continue to loop through urls until we have scraped all valid
# needed a while loop so we can modify size of urls through process
while (len(urls) > 0) and ctr < 2:    #(len(done_urls) < 200):
    
    # timer for performance across url loops
    forStart = time.time()
    
    # go through every url that has not been scraped yet
    for url in urls:
        print("URL in for loop: ", url)
        
        # timer for performance
        start = time.time()
        
        # governor to avoid being banned/blocked
        time.sleep(5)   # time governor
        
        # check for email and run all other links through scrape
        html = sc.scrapeHTML(url)
        
        # end timer for performance and print status
        end = time.time()
        print(f'Elapsed time: {end - start}')
        
        # validate html and then add to the pagelinks all the links that were scraped
        if html: 
            #print(f'Html scraped: {html[:50]}')
            data = {
                "url": url, 
                "html": html, 
                }

            # insert dict into db document
            htmlCollection.insert_one(data)
            
            # record url in "done" collection
            done_urls.add(url)
            
            pagelinks = pagelinks.union(sc.scrapeURL(html, url))
            print(f'Extracted links: {len(pagelinks)}')
        else:
            print(f'Main Loop: HTML not extracted')
        
        ####################################################
        # TODO: update done_urls, external and urls to CSV #
        ####################################################
        
    # end timer of for loop
    forEnd = time.time()
    
    # print status for testing
    print()
    print(f'Number of Urls: {len(urls)}')
    print(f'Process Time: {forEnd -forStart}')
    print('--------------------------------------------------------------')
        
        
    
    # add links from page to urls set outside of for loop
    if html:
        for link in pagelinks:
            if domain in link:
                urls.add(link)
                # print(f"Link in pagelinks: {link}")
            else:
                external_urls.append(link)
    
    # remove done_urls from todo urls and increment counter
    urls -= done_urls
    ctr += 1
    
    # print status for testing
    print("done_urls: ", len(done_urls))
    print("(urls - done_urls): ", len(urls))
    print("LOOP counter: ", ctr)
    print("--------------------")


URL in for loop:  https://billypenn.com/
Elapsed time: 5.338198661804199
Extracted links: 78

Number of Urls: 1
Process Time: {forEnd -forStart}
--------------------------------------------------------------
done_urls:  1
(urls - done_urls):  51
LOOP counter:  1
--------------------
URL in for loop:  https://billypenn.com/2018/06/14/rip-big-al-meltzer-philly-sports-broadcasting-legend/
Elapsed time: 5.229690313339233
Extracted links: 88
URL in for loop:  https://billypenn.com/2018/06/12/this-housing-project-in-philly-helps-residents-double-their-income/
Elapsed time: 5.243704319000244
Extracted links: 120
URL in for loop:  https://billypenn.com/advertising/
Elapsed time: 5.332225799560547
Extracted links: 121
URL in for loop:  https://billypenn.com/stories/
Elapsed time: 5.27692437171936
Extracted links: 145
URL in for loop:  https://billypenn.com/2018/06/10/philly-pride-photos-30-frank-fierce-and-fabulous-moments/
Elapsed time: 5.248501539230347
Extracted links: 183
URL in for loop:  

In [ ]:
sc.scrapeHTML("mailto:mhager3@hotmail.com", htmlCollection, done_urls)